In [1]:
import os
import requests
import tabula
import pandas as pd
import re
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
TABLE = 'Tuition'

In [2]:
def download_pdf_file(url: str) -> bool:
    """Download PDF from given URL to local directory.

    :param url: The url of the PDF file to be downloaded
    :return: True if PDF file was successfully downloaded, otherwise False.
    """

    response = requests.get(url, stream=True)

    if response.status_code == 200:
        with open(TABLE +'.pdf', 'wb') as pdf_object:
            pdf_object.write(response.content)
            print(f'{TABLE} was successfully saved!')
            return True
    else:
        print(f'Uh oh! Could not download {TABLE},')
        print(f'HTTP response status code: {response.status_code}')
        return False


URL = 'https://www.monroecollege.edu/sites/default/files/documents/2024/01/29/Graduate_tuition_and_fees_2024_2025.pdf'
download_pdf_file(URL)

Tuition was successfully saved!


True

In [3]:
dfs = tabula.read_pdf(TABLE + '.pdf', pages='all')
df_first_table = dfs[0]
df_first_table.to_csv(TABLE + '_RAW.csv', index=False)

Mar 21, 2024 12:55:19 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
/Users/bilalakhtar/PycharmProjects/School-Info2/.venv1/lib/python3.11/site-packages/tabula/io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [4]:
df = pd.read_csv(TABLE + '_RAW.csv')

In [5]:
df.columns = range(df.shape[1])

In [6]:
df.drop(df.columns[4], axis=1, inplace=True)

In [7]:
df[1] = df[1].apply(lambda x: x + ' and Fees' if str(x) == 'Full time Tuition' else x)

In [8]:
df[2] = df[2].apply(lambda x: x + ' Credit' if str(x) == 'Cost Per' else x)

In [9]:
df.loc[df[1] == 'Full time Tuition and Fees', 2] = 'Cost Per Credit'

In [10]:
df[3] = df[3].apply(lambda x: x + ' Class' if str(x) == 'Cost Per' else x)

In [11]:
df[2] = df[2].combine_first(df[3])
df.drop(df.columns[3], axis=1, inplace=True)

In [12]:
df = df.loc[~df[0].isin(["and Fees", "Credit", "Class"])]

In [18]:
df

,0,1,2
0,Application Fees,NaN,Total Cost
1,Graduate Admissions Fee (Nonrefundable),NaN,$50
2,Acceptance Confirmation Fee (International Students),NaN,$100
3,Graduate Tuition (Full Time = 7 credits or above),Full time Tuition and Fees,Cost Per Credit
6,First U.S. master’s degree (7 credits)*,"$6,529",$944
7,First U.S. master’s degree (9 credits),"$9,426",$944
8,Second Master’s Degree – First Master’s Degree earned from,NaN,NaN
9,Monroe College (7 credits)*,"$4,078",$540
10,Second Master’s Degree – First Master’s Degree earned from,NaN,NaN
11,Monroe College (9 credits),"$5,816",$554


In [19]:
df.to_csv(TABLE + '.csv')